In [1]:
import dataloader_t2i as dataloader
import transformers

tokenizer  = transformers.AutoTokenizer.from_pretrained(
        '/home/MaTianren/Workspace/LLaVA-15/checkpoints/lang-mirage-7b-lora-continuv-merged', use_fast=False)
tokenizer.chat_template = 'tokenizer'

/home/MaTianren/anaconda3/envs/qwen/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenized_dataset = dataloader.get_dataset(
    dataset_name='gsm8k', tokenizer=tokenizer)


In [18]:
import torch
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    text_embeds = [torch.tensor(x['text_embeds'][0]) for x in batch]
    image_tokens = [torch.tensor(x['image_tokens'][0]) for x in batch]
    tokens = {
        'text_embeds': pad_sequence(text_embeds, batch_first=True, padding_value=0),
        'image_tokens': pad_sequence(image_tokens, batch_first=True, padding_value=0)
        # 'text_embeds': text_embeds,
        # 'image_tokens': image_tokens
    }
    return tokens

train_set = dataloader.get_dataset(
    dataset_name='gsm8k', tokenizer=tokenizer)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=4,
    num_workers=8,
    collate_fn=collate_fn,
    drop_last=True,
    persistent_workers=True)

In [19]:
for batch in train_loader:
    x1 = batch['text_embeds'][0]
    x2 = batch['image_tokens'][0]
    break
print(x1.shape)
print(x2.shape)

torch.Size([114, 2048])
torch.Size([256])


In [10]:
x1.shape

torch.Size([1, 110, 2048])